In [142]:
from typing import Any
import json
import os
import re
from pprint import pprint
from warnings import simplefilter
import numpy as np
import pandas as pd
from cleantext import clean

from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_validate

simplefilter("ignore")
pd.set_option("display.max_columns", 0)

### 1. Loading and Inspecting the Data

When you create a notebook within a Kaggle competition, the competition's data is automatically attached and available in the `../input/` directory.

The dataset is in a `.jsonl` format, which means each line is a separate JSON object. This is great because we can process it one line at a time without needing to load the entire large file into memory.

Let's write a simple loop to load the training data and inspect the first battle.

In [143]:
COMPETITION_NAME = 'fds-pokemon-battles-prediction-2025'
DATA_PATH = os.getcwd() #os.path.join('../input', COMPETITION_NAME)
UNTOUCHED = {'battle_id', 'player_won'}
INFORMATIVE = {
    "p1_unique_pokemon",
    "p2_unique_pokemon",
    "final_p1_hp",  
    "p1_fainted_count", 
    "p1_turns_statused", 
    "p1_missed_turns", 
    "p2_turns_statused",
    "p2_missed_turns", 
    "battle_id",
    "player_won"
}

train_file_path = os.path.join(DATA_PATH, 'train.jsonl')
test_file_path = os.path.join(DATA_PATH, 'test.jsonl')

print(f"Loading data from '{train_file_path}'...")
try:
    with open(train_file_path, 'r', encoding="utf-8") as f:
        train_data = [json.loads(line) for line in f]

    print(f"Successfully loaded {len(train_data)} battles.")

    #print("\n--- Structure of the first train battle: ---")
    if train_data:
        first_battle = train_data[0]
        
        battle_for_display = first_battle.copy()
        battle_for_display['battle_timeline'] = battle_for_display.get('battle_timeline', [])[:2] # Show first 2 turns
        
        #pprint(battle_for_display)
        if len(first_battle.get('battle_timeline', [])) > 3:
            print("    ...")
            print("    (battle_timeline has been truncated for display)")

except FileNotFoundError:
    print(f"ERROR: Could not find the training file at '{train_file_path}'.")
    print("Please make sure you have added the competition data to this notebook.")

Loading data from 'c:\Users\stefa\PycharmProjects\pokemon-challenge\train.jsonl'...
Successfully loaded 10000 battles.
    ...
    (battle_timeline has been truncated for display)


### 2. Basic Feature Engineering

A successful model will likely require creating many complex features. For this starter notebook, however, we will create a very simple feature set based **only on the initial team stats**. This will be enough to train a model and generate a submission file.

It's up to you to engineer more powerful features!

In [144]:
def features_check(data: dict) -> None:
    print("All battles have at least one turn: ", all(all(turn for turn in battle.get('battle_timeline', False)) for battle in data))
    print("All battles' turns have at least one P1 move: ", 
        all((
            any((turn.get("p1_move_details", False) for turn in battle.get('battle_timeline', False))) for battle in data
        ))
    )
    print("All battles' turns have at least one P2 move: ", 
        all((
            any((turn.get("p2_move_details", False) for turn in battle.get('battle_timeline', False))) for battle in data
        ))
    )
    print("player_won feature always exists: ", all(('player_won' in battle for battle in data)))
    print("P1 Team always exists: ", all(battle.get('p1_team_details', False) for battle in data))
    print("P2 Team always exists: ", all(battle.get('p2_team_details', False) for battle in data))
    
    return None

In [145]:
# ------------------------------------------------------------
# Aggregate collected per-player stats
# ------------------------------------------------------------
def agg_pokemons_stats(prefix: str, stats: dict[str, Any]):
    return {
        f"{prefix}_mean_power": np.mean(stats["powers"]) if stats["powers"] else 0,
        f"{prefix}_mean_accuracy": np.mean(stats["accuracy"]) if stats["accuracy"] else 0,
        f"{prefix}_lost_hp": stats["lost_hp"],
        f"{prefix}_turns_statused": stats["turns_statused"],
        f"{prefix}_missed_turns": stats["missed_turns"],
        f"{prefix}_switches": stats["switches"],
        f"{prefix}_net_boost": stats["net_boost"],
    }

In [ ]:
# ------------------------------------------------------------
# Main Feature Engineering Function
# ------------------------------------------------------------
def create_features(data: list[dict]) -> pd.DataFrame:  # Function takes a list of battle dicts and returns a pandas DataFrame
    # Generates a comprehensive feature set for each battle
    # The function derives team-level aggregates, lead-pokemon attributes, timeline-derived dynamics, HP/status/effect/boost tracking, per-pokemon presence flags, and more

    # ------------------------------------------------------------
    # Collect all unique Player-1 Pokémon names across dataset
    # This allows creating consistent one-hot "p1_has_<pkm>" features.
    # ------------------------------------------------------------
    unique_p1_names = set()  # Set to accumulate every distinct p1 Pokémon name observed in the dataset
    for battle in data:  # Iterate each battle record (each element is expected to be a dict)
        for p in battle.get('p1_team_details', []) or []:  # Iterate the declared team details for player 1 (if any)
            if p and 'name' in p: unique_p1_names.add(p['name'])  # Add pokemon name if present to ensure presence features

        for timeline_entry in battle.get('battle_timeline', []) or []:  # Iterate timeline to capture leads introduced only in timeline
            if name := timeline_entry.get('p1_pokemon_state', {}).get('name'):  # Use walrus operator to capture name if it exists, and if so, adds it to the pokemon names set
                unique_p1_names.add(name)

    unique_name_to_feat = {  # Map raw Pokémon name with its respective feature column name (one-hot)
        name: f"p1_has_{clean(name, lower=True, normalize_whitespace=True)}" 
        for name in unique_p1_names
    }  # clean() normalizes names for safe column keys (lowercase, trim spaces, etc.)

    # Feature rows
    feature_list = []  # List that will collect a dict of features per battle, converted to DataFrame and returned at the end by the function

    # ------------------------------------------------------------
    # Process each battle
    # ------------------------------------------------------------
    for battle in data:  # Loop over battles again to build a dataframe record for each battle
        features = {}  # Dict to accumulate features for the current battle

        # Initialize all p1_has_<pkm> = False by default
        features.update({feat_key: False for feat_key in unique_name_to_feat.values()}) # For each pokemon we set by default its non-presence in the battle

        # --------------------------------------------------------
        # (A) TEAM-LEVEL FEATURES
        # --------------------------------------------------------
        if p1_team := battle.get('p1_team_details', []):  # If team details exist, compute aggregated base-stat features and presence flags
            # Mark pokemons' presence features
            for p in p1_team:
                if p and 'name' in p and p['name'] in unique_name_to_feat: # If the pokemon name is within the names set we created before then set one-hot True for Pokémon that are on p1 team
                    features[unique_name_to_feat[p['name']]] = True

            # Compute team base-stat means — use numpy nanmean to be robust to missing or zero entries
            features.update({
                'p1_mean_hp': np.nanmean([p.get('base_hp', 0)  for p in p1_team]), # Mean base HP for p1 team
                'p1_mean_spe': np.nanmean([p.get('base_spe', 0) for p in p1_team]), # Mean base Speed
                'p1_mean_atk': np.nanmean([p.get('base_atk', 0) for p in p1_team]), # Mean base Attack
                'p1_mean_def': np.nanmean([p.get('base_def', 0) for p in p1_team]), # Mean base Defense
                'p1_mean_special':np.nanmean([p.get('base_spa', 0) for p in p1_team]), # Mean base Special Attack
            })

        else:
            # Default zero-values if team structure is not provided to avoid missing columns downstream
            features.update({
                'p1_mean_hp': 0,
                'p1_mean_spe': 0,
                'p1_mean_atk': 0,
                'p1_mean_def': 0,
                'p1_mean_special': 0,
            })

        # --------------------------------------------------------
        # (B) LEAD-POKÉMON FEATURES
        # --------------------------------------------------------
        timeline = battle.get("battle_timeline", [])  # timeline is the turn-by-turn snapshot list (may be empty)
        first_turn = timeline[0] if timeline else {}  # first_turn is used to infer initial lead Pokémon

        p1_lead = first_turn.get("p1_pokemon_state", {}).get("name", "")  # p1 lead pokemon name (empty string if unknown)
        p2_lead_details = battle.get("p2_lead_details", {})  # p2 lead pokemon details may be provided separately in some exports

        # Extract P1 lead stats by searching the declared team for the lead's base stats
        p1_lead_stats = {}  # Start with empty stats; populate only if lead is found in p1_team
        for p in p1_team:
            if p.get('name') == p1_lead:
                p1_lead_stats = {
                    'p1_lead_hp': p.get('base_hp', 0),  # Base HP of lead
                    'p1_lead_atk': p.get('base_atk', 0),  # Base Attack of lead
                    'p1_lead_def': p.get('base_def', 0),  # Base Defense of lead
                    'p1_lead_special': p.get('base_spa', 0),  # Base Special Attack of lead
                    'p1_lead_spe': p.get('base_spe', 0),  # Base Speed of lead
                }
                break  # Stop searching once we found the lead entry
        features.update(p1_lead_stats)  # Insert found lead pokemon stats into the features dict

        # P2 lead stats — if external details provided, use them, also compute simple lead speed advantage flag
        if p2_lead_details:
            features['p2_lead_hp'] = p2_lead_details.get('base_hp', 0)  # p2 lead pokemon base HP (fallback 0)
            features['p2_lead_spe'] = p2_lead_details.get('base_spe', 0)  # p2 lead pokemon speed
            features['p2_lead_atk'] = p2_lead_details.get('base_atk', 0)  # p2 lead pokemon attack
            features['p2_lead_def'] = p2_lead_details.get('base_def', 0)  # p2 lead pokemon defense
            features['p2_lead_special'] = p2_lead_details.get('base_spa', 0)  # p2 lead pokemon special attack
            # spe_lead_adv is 1 if p1 lead's base speed strictly greater than p2 lead pokemon's base speed, else 0
            features['spe_lead_adv'] = int(p1_lead_stats.get('p1_lead_spe', 0) > p2_lead_details.get('base_spe', 0))
        else:
            features['spe_lead_adv'] = 0  # Default to no advantage if p2 lead details aren't available

        # --------------------------------------------------------
        # (C) TIMELINE-BASED FEATURES
        # --------------------------------------------------------
        # Initialize per-player dynamic stats to keep per-turn aggregates organized and consistent
        def init_stats():  # init_stats() returns a tracking dict template for a player's dynamic values through the timeline
            return {
                "powers": [],  # List of move base powers seen over timeline (for damage potential)
                "accuracy": [],  # List of move accuracies attempted
                "hp_t0": {},  # Map pokemon_name to the latest observed hp_pct (for per-pokemon hp tracking)
                "lost_hp": 0,  # Accumulated hp% lost across all Pokémon (sum of deltas)
                "turns_statused": 0,  # Total turns player had any status (e.g., brn, psn, par)
                "missed_turns": 0,  # Number of turns with no move details (could indicate skip or missing data)
                "priority": 0,  #Placeholder for priority move use counts (not filled here but reserved)
                "switches": 0,  # Count of in-battle Pokémon switches observed
                "net_boost": 0,  # Cumulative net boosts across tracked stats
                "base_boosts": {k: 0 for k in ["atk", "def", "spa", "spd", "spe"]},  # Last-observed boost baseline
                "status_turns": {},  # Map status to the total turns that status was active on any pokemon
                "effect_turns": {},  # Map effect to the total turns that effect was active (e.g., substitute)
                "hp_loss_while_statused": 0,  # HP lost while a status was active (to measure status impact)
                "hp_loss_while_effect": 0,  # HP lost while effects were active (to measure effect damage)
            }

        p1_stats = init_stats()  # Initialize dynamic stats tracking dict for player 1
        p2_stats = init_stats()  # Initialize dynamic stats tracking dict for player 2

        if timeline:  # Only populate timeline-derived features if we actually have per-turn snapshots
            p1_names, p2_names = [], []  # Lists to record the sequence of Pokémon names seen for each player
            p2_hp_deltas = []  # List of HP percentage drops applied to p2 between consecutive turns (for mean damage)

            prev_p1_hp, prev_p2_hp = None, None  # Previous-turn hp_pct for p1 and p2 (used to compute instantaneous loss)
            prev_p1_status, prev_p2_status = None, None  # Previous-turn status strings (used to attribute loss to statuses)
            prev_p1_effects, prev_p2_effects = set(), set()  # Previous-turn effects sets (used to attribute loss to effects)

            # Iterate over turns
            for turn_idx, turn in enumerate(timeline):  # Loop through timeline with index to allow comparing to previous turn
                p1_state = turn.get("p1_pokemon_state", {})  # Snapshot dict for p1's active pokemon state at this turn
                p2_state = turn.get("p2_pokemon_state", {})  # Snapshot dict for p2's active pokemon state at this turn

                p1_name = p1_state.get("name", "")  # Active p1 pokemon name for this turn (empty string fallback)
                p2_name = p2_state.get("name", "")  # Active p2 pokemon name for this turn (empty string fallback)
                p1_hp = p1_state.get("hp_pct", 1.0)  # Active p1 hp as a fraction of max (default 1.0 if missing)
                p2_hp = p2_state.get("hp_pct", 1.0)  # Active p2 hp as a fraction of max
                p1_status = p1_state.get("status", "nostatus")  # Status string or "nostatus"
                p2_status = p2_state.get("status", "nostatus")  # Status string or "nostatus"
                p1_effects = set(p1_state.get("effects", ["noeffect"]))  # Convert effects list into a set
                p2_effects = set(p2_state.get("effects", ["noeffect"]))  # Do the same for p2

                # Track pokemon identity appearances
                if p1_name:
                    p1_names.append(p1_name)  # Append to p1 seen pokemons sequence for tracking
                    if p1_name in unique_name_to_feat:
                        features[unique_name_to_feat[p1_name]] = True  # Turn on one-hot if this pokemon appears at any turn
                if p2_name:
                    p2_names.append(p2_name)  # Append to p2 seen pokemons sequence

                # --------------------------------------------------
                # MOVES AND ACCURACY
                # --------------------------------------------------
                for key, stats in [
                    ("p1_move_details", p1_stats),
                    ("p2_move_details", p2_stats)
                ]:  # iterate both players' move detail keys with corresponding stat accumulator
                    move = turn.get(key)  # grab move details dict for this turn (if present)
                    if move:
                        stats["powers"].append(move.get("base_power", 0))  # record base power (0 if moves like status)
                        stats["accuracy"].append(move.get("accuracy", 0))  # record accuracy metric when present
                    else:
                        stats["missed_turns"] += 1  # increment missed_turns where move info is absent (possible skip or missing data)

                # --------------------------------------------------
                # STATUS / EFFECT TRACKING
                # --------------------------------------------------
                if p1_status != "nostatus":
                    p1_stats["turns_statused"] += 1  # increment aggregate count of turns p1 had any non-empty status
                    p1_stats["status_turns"][p1_status] = p1_stats["status_turns"].get(p1_status, 0) + 1  # per-status counter
                if p2_status != "nostatus":
                    p2_stats["turns_statused"] += 1  # same for p2
                    p2_stats["status_turns"][p2_status] = p2_stats["status_turns"].get(p2_status, 0) + 1

                for effect in p1_effects:
                    p1_stats["effect_turns"][effect] = p1_stats["effect_turns"].get(effect, 0) + 1  # accumulate effect usage counts
                for effect in p2_effects:
                    p2_stats["effect_turns"][effect] = p2_stats["effect_turns"].get(effect, 0) + 1

                # --------------------------------------------------
                # HP TRACKING
                # --------------------------------------------------
                if p1_name in p1_stats["hp_t0"]:
                    delta = p1_stats["hp_t0"][p1_name] - p1_hp  # delta = previous observed hp_pct - current hp_pct
                    if delta > 0:
                        p1_stats["lost_hp"] += delta  # add to cumulative lost_hp only when actual loss occurred
                p1_stats["hp_t0"][p1_name] = p1_hp  # update the last-observed hp_pct for this pokemon

                if p2_name in p2_stats["hp_t0"]:
                    delta = p2_stats["hp_t0"][p2_name] - p2_hp
                    if delta > 0:
                        p2_stats["lost_hp"] += delta  # similar accumulation for p2
                p2_stats["hp_t0"][p2_name] = p2_hp

                # HP loss under status/effect: attribute previous-turn → current loss to status/effect active in previous turn
                if prev_p1_hp is not None and p1_hp < prev_p1_hp:
                    loss = prev_p1_hp - p1_hp  # compute amount lost since previous snapshot
                    if prev_p1_status != "nostatus":
                        p1_stats["hp_loss_while_statused"] += loss  # add to status-attributed loss if previous had status
                    if prev_p1_effects:
                        p1_stats["hp_loss_while_effect"] += loss  # likewise for effect-attributed loss

                if prev_p2_hp is not None and p2_hp < prev_p2_hp:
                    loss = prev_p2_hp - p2_hp
                    if prev_p2_status != "nostatus":
                        p2_stats["hp_loss_while_statused"] += loss
                    if prev_p2_effects:
                        p2_stats["hp_loss_while_effect"] += loss

                # --------------------------------------------------
                # SWITCHES
                # --------------------------------------------------
                if turn_idx > 0:  # Avoid counting the first turn as a switch (initial send-out isn't a 'switch' event)
                    if p1_name != timeline[turn_idx - 1]['p1_pokemon_state']['name']:
                        p1_stats["switches"] += 1  # increment when the active p1 pokemon name differs from previous turn
                    if p2_name != timeline[turn_idx - 1]['p2_pokemon_state']['name']:
                        p2_stats["switches"] += 1  # same logic for p2

                # --------------------------------------------------
                # BOOST TRACKING
                # --------------------------------------------------
                p1_boosts = p1_state.get("boosts", {})  # dictionary of stat boosts this turn for p1 active pokemon
                p2_boosts = p2_state.get("boosts", {})  # same for p2

                for stat in ["atk", "def", "spa", "spd", "spe"]:
                    # net_boost accumulates changes relative to the last baseline stored in base_boosts
                    p1_stats["net_boost"] += p1_boosts.get(stat, 0) - p1_stats["base_boosts"].get(stat, 0)
                    p2_stats["net_boost"] += p2_boosts.get(stat, 0) - p2_stats["base_boosts"].get(stat, 0)

                # Store deltas for mean-damage computation: compute p2 hp drop relative to previous turn's p2 hp
                if turn_idx > 0:
                    prev_hp = timeline[turn_idx - 1]['p2_pokemon_state']['hp_pct']  # previous p2 hp_pct snapshot
                    p2_hp_deltas.append(prev_hp - p2_hp)  # append observed delta (can be negative if healed)

                # Update previous values for the next-iteration comparisons
                prev_p1_hp = p1_hp  # set previous hp for next loop iteration
                prev_p2_hp = p2_hp
                prev_p1_status = p1_status  # remember previous status to attribute hp loss next turn
                prev_p2_status = p2_status
                prev_p1_effects = p1_effects  # remember previous effects set
                prev_p2_effects = p2_effects
                p1_stats["base_boosts"] = p1_boosts.copy()  # update baseline boosts to the current turn's boosts
                p2_stats["base_boosts"] = p2_boosts.copy()

            # --------------------------------------------------------
            # Aggregate summary timeline features
            # --------------------------------------------------------
            features['p1_unique_pokemon'] = len(set(p1_names))  # number of distinct p1 pokémon seen during the battle
            features['p2_unique_pokemon'] = len(set(p2_names))  # number of distinct p2 pokémon seen during the battle
            features['n_turns'] = len(timeline)  # total number of turns / snapshots recorded in timeline
            # mean_damage_dealt: mean of positive deltas (damage dealt to p2 between consecutive turns), 0 if no data
            features['mean_damage_dealt'] = np.nanmean([d for d in p2_hp_deltas if d > 0]) if p2_hp_deltas else 0

            features['final_p1_hp'] = timeline[-1]['p1_pokemon_state'].get('hp_pct', None)  # final active p1 hp_pct observed
            features['p1_fainted_count'] = sum(t['p1_pokemon_state']['status'] == 'fnt' for t in timeline)  # count fnt occurrences
            features['p2_fainted_count'] = sum(t['p2_pokemon_state']['status'] == 'fnt' for t in timeline)

            # HP final states: average of last-observed hp_pct for all seen pokémon of each player
            p1_final_hp = np.nanmean(list(p1_stats["hp_t0"].values())) if p1_stats["hp_t0"] else 1.0  # fallback 1.0 full hp
            p2_final_hp = np.nanmean(list(p2_stats["hp_t0"].values())) if p2_stats["hp_t0"] else 1.0

            features['p1_mean_final_hp'] = p1_final_hp  # per-player mean of final hp% across their pokémon
            features['p2_mean_final_hp'] = p2_final_hp

            # HP-based ratios: normalized measures to compare damage taken vs remaining HP (eps added for numerical stability)
            features['p1_hp_loss_ratio'] = p1_stats["lost_hp"] / (p1_stats["lost_hp"] + p1_final_hp + 1e-6)
            features['p2_hp_loss_ratio'] = p2_stats["lost_hp"] / (p2_stats["lost_hp"] + p2_final_hp + 1e-6)
            features['hp_diff_lost'] = p2_stats["lost_hp"] - p1_stats["lost_hp"]  # absolute difference in hp lost totals
            features['hp_diff_final'] = p1_final_hp - p2_final_hp  # difference in mean final hp% (p1 - p2)
            features['hp_ratio_p1_p2'] = p1_stats["lost_hp"] / (p2_stats["lost_hp"] + 1e-6)  # ratio of lost hp between players

            # HP stats: mean and min-level snapshots across pokemon observed
            features['p1_mean_hp_pct'] = np.mean([v for v in p1_stats["hp_t0"].values() if v > 0]) if p1_stats["hp_t0"] else 1.0
            features['p2_mean_hp_pct'] = np.mean([v for v in p2_stats["hp_t0"].values() if v > 0]) if p2_stats["hp_t0"] else 1.0
            features['p1_min_hp'] = np.min(list(p1_stats["hp_t0"].values())) if p1_stats["hp_t0"] else 1.0
            features['p2_min_hp'] = np.min(list(p2_stats["hp_t0"].values())) if p2_stats["hp_t0"] else 1.0

            # Survival: fraction of unique pokémon that remain with hp > 0 at last observation
            p1_alive = sum(1 for v in p1_stats["hp_t0"].values() if v > 0)  # count alive p1 pokemon at final snapshot
            p2_alive = sum(1 for v in p2_stats["hp_t0"].values() if v > 0)
            features['p1_survival_ratio'] = p1_alive / (len(p1_stats["hp_t0"]) + 1e-6)  # divide by total observed pkm count
            features['p2_survival_ratio'] = p2_alive / (len(p2_stats["hp_t0"]) + 1e-6)
            features['survival_diff'] = features['p1_survival_ratio'] - features['p2_survival_ratio']  # comparative survival

            # HP per Pokémon / per turn normalized metrics for learning algorithms
            features['p1_avg_hp_lost_per_pkm'] = p1_stats["lost_hp"] / (len(p1_stats["hp_t0"]) + 1e-6)
            features['p2_avg_hp_lost_per_pkm'] = p2_stats["lost_hp"] / (len(p2_stats["hp_t0"]) + 1e-6)
            features['p1_hp_loss_per_turn'] = p1_stats["lost_hp"] / (features['n_turns'] + 1e-6)  # avg lost hp per turn
            features['p2_hp_loss_per_turn'] = p2_stats["lost_hp"] / (features['n_turns'] + 1e-6)
            features['hp_delta_per_turn'] = (p2_stats["lost_hp"] - p1_stats["lost_hp"]) / (features['n_turns'] + 1e-6)
            # hp_relative_strength: normalized difference in final hp averages (signed), small eps avoids div-by-zero
            features['hp_relative_strength'] = (p1_final_hp - p2_final_hp) / (abs(p1_final_hp) + abs(p2_final_hp) + 1e-6)

            # EFFECT/STATUS IMPACTS: ratios describing how much hp loss was attributable to status/effect conditions
            features['p1_hp_loss_status_ratio'] = p1_stats["hp_loss_while_statused"] / (p1_stats["lost_hp"] + 1e-6)
            features['p2_hp_loss_status_ratio'] = p2_stats["hp_loss_while_statused"] / (p2_stats["lost_hp"] + 1e-6)
            features['p1_hp_loss_effect_ratio'] = p1_stats["hp_loss_while_effect"] / (p1_stats["lost_hp"] + 1e-6)
            features['p2_hp_loss_effect_ratio'] = p2_stats["hp_loss_while_effect"] / (p2_stats["lost_hp"] + 1e-6)

            features['hp_diff_statused'] = p2_stats["hp_loss_while_statused"] - p1_stats["hp_loss_while_statused"]  # net status-attributed loss diff
            features['hp_diff_effected'] = p2_stats["hp_loss_while_effect"] - p1_stats["hp_loss_while_effect"]  # net effect-attributed diff

            features['p1_resistance_index'] = 1 - features['p1_hp_loss_status_ratio']  # interpret as fraction of hp not lost to status
            features['p2_resistance_index'] = 1 - features['p2_hp_loss_status_ratio']
            features['resistance_diff'] = features['p1_resistance_index'] - features['p2_resistance_index']  # p1 minus p2 resistance measure

            # Add individual status/effect counts as explicit features (sparse, but informative)
            features.update({f"p1_pkm_{k}": v for k, v in p1_stats["status_turns"].items()}) # e.g., p1_pkm_brn = number of turns any p1 pokemon was burned
            features.update({f"p2_pkm_{k}": v for k, v in p2_stats["status_turns"].items()})
            features.update({f"p1_pkm_{k}": v for k, v in p1_stats["effect_turns"].items()}) # e.g., p1_pkm_substitute = number of turns substitute was active
            features.update({f"p2_pkm_{k}": v for k, v in p2_stats["effect_turns"].items()})


        else:
            # No timeline → fill minimal fields with conservative defaults to keep consistent schema
            features.update({
                'p1_unique_pokemon': 0,
                'p2_unique_pokemon': 0,
                'n_turns': 0,
                'mean_damage_dealt': 0,
                'final_p1_hp': None,
                'p1_fainted_count': 0,
                'p2_fainted_count': 0,
            })

        # Append aggregated stats computed per-pokemon (helper function expected to return a dict of additional features)
        features.update(agg_pokemons_stats("p1", p1_stats))  # merge p1 aggregated per-pokemon stats (function must be defined elsewhere)
        features.update(agg_pokemons_stats("p2", p2_stats))  # merge p2 aggregated per-pokemon stats

        # Outcome + ID
        features['battle_id'] = battle.get('battle_id')  # preserve original battle identifier for traceability
        if 'player_won' in battle:
            features['player_won'] = int(battle['player_won'])  # convert boolean to integer label (1/0) for modelling

        feature_list.append(features)  # add this battle's feature dict to the master list

    return pd.DataFrame(feature_list).fillna(0)  # construct final DataFrame and fill remaining NaNs with zeros for model compatibility


In [147]:
print("Processing training data...")
train_df = create_features(train_data)

print("\nProcessing test data...")
with open(test_file_path, 'r', encoding="utf-8") as f:
    test_data = [json.loads(line) for line in f]
        
test_df = create_features(test_data)

Processing training data...

Processing test data...


In [148]:
keepers = list(set(train_df.columns).difference(UNTOUCHED)) #Keeping only the features which need to be scaled

In [149]:
#Scaling features
scaler = StandardScaler(with_mean=True, with_std=True)

train_df[keepers] = scaler.fit_transform(train_df[keepers])
test_df[keepers] = scaler.fit_transform(test_df[keepers])

In [150]:
print("\nTraining dataset preview:")
display(train_df.head())
display(train_df.describe())
display(train_df.dtypes)

print("\nTesting dataset preview:")
display(test_df.head())
display(test_df.describe())
display(test_df.dtypes)

print(list(test_df.columns))


Training dataset preview:


,p1_has_alakazam,p1_has_golem,p1_has_slowbro,p1_has_zapdos,p1_has_snorlax,p1_has_cloyster,p1_has_dragonite,p1_has_persian,p1_has_charizard,p1_has_chansey,p1_has_gengar,p1_has_jynx,p1_has_rhydon,p1_has_starmie,p1_has_tauros,p1_has_exeggutor,p1_has_articuno,p1_has_lapras,p1_has_jolteon,p1_has_victreebel,p1_mean_hp,p1_mean_spe,p1_mean_atk,p1_mean_def,p1_mean_special,p1_lead_hp,p1_lead_atk,p1_lead_def,p1_lead_special,p1_lead_spe,p2_lead_hp,p2_lead_spe,p2_lead_atk,p2_lead_def,p2_lead_special,spe_lead_adv,p1_unique_pokemon,p2_unique_pokemon,n_turns,mean_damage_dealt,...,p2_pkm_par,p1_pkm_noeffect,p1_pkm_reflect,p2_pkm_noeffect,p1_mean_power,p1_mean_accuracy,p1_lost_hp,p1_turns_statused,p1_missed_turns,p1_switches,p1_net_boost,p2_mean_power,p2_mean_accuracy,p2_lost_hp,p2_turns_statused,p2_missed_turns,p2_switches,p2_net_boost,battle_id,player_won,p2_pkm_substitute,p2_pkm_reflect,p1_pkm_frz,p1_pkm_wrap,p1_pkm_tox,p1_pkm_psn,p1_pkm_confusion,p1_pkm_substitute,p2_pkm_confusion,p1_pkm_clamp,p2_pkm_clamp,p2_pkm_tox,p2_pkm_psn,p2_pkm_brn,p2_pkm_wrap,p1_pkm_brn,p1_pkm_typechange,p2_pkm_typechange,p1_pkm_firespin,p2_pkm_firespin
0,1.051737,-0.371718,-0.343246,-0.603074,0.445173,-0.402467,-0.199834,-0.187036,-0.09259,0.42403,-0.576273,-0.559949,-0.595522,1.082819,0.341423,0.573809,-0.193247,-0.341606,-0.317357,-0.202524,0.202093,0.520813,-0.732064,-0.745443,0.644639,-0.508230,0.553031,1.039088,-0.562587,0.763073,-0.279683,0.490498,0.642099,1.185993,-0.741369,-0.747885,-1.404495,-1.331840,0.0,-0.300490,...,-0.639394,0.070662,0.027346,0.532870,-0.118412,-0.823055,-0.564623,-1.089872,-1.109216,-0.871133,-0.114718,0.446467,0.785598,0.029295,1.042992,2.443488,0.622462,-0.136512,0,1,-0.166375,-0.464671,-0.32638,-0.041399,-0.105891,-0.071695,-0.168464,-0.173161,-0.16849,-0.049726,-0.052151,-0.108606,-0.079622,-0.06164,-0.041126,-0.064922,-0.014144,-0.014144,-0.011768,-0.010946
1,-0.950808,-0.371718,2.913360,-0.603074,0.445173,-0.402467,-0.199834,-0.187036,-0.09259,0.42403,-0.576273,1.785878,-0.595522,-0.923516,0.341423,0.573809,-0.193247,-0.341606,-0.317357,-0.202524,0.761596,-1.738011,-0.732064,-0.492591,-0.759727,-0.429812,-0.327042,-0.661341,-0.806141,0.078764,-0.450188,0.723525,-0.711192,-0.611967,1.072092,-0.747885,0.918520,0.950271,0.0,-1.373696,...,-0.993334,0.549401,-0.484271,0.532870,1.450473,0.172684,-0.085086,-0.341616,0.192108,0.586708,-0.114718,0.170319,0.322559,0.539208,-1.090316,0.158909,0.622462,-1.382055,1,1,-0.166375,-0.464671,-0.32638,-0.041399,-0.105891,-0.071695,-0.168464,-0.173161,-0.16849,-0.049726,-0.052151,-0.108606,-0.079622,-0.06164,-0.041126,-0.064922,-0.014144,-0.014144,-0.011768,-0.010946
2,-0.950808,-0.371718,-0.343246,-0.603074,0.445173,-0.402467,-0.199834,-0.187036,-0.09259,0.42403,1.735290,-0.559949,1.679199,-0.923516,0.341423,0.573809,-0.193247,-0.341606,-0.317357,-0.202524,0.823762,-1.224642,0.906915,0.097399,-0.759727,0.040699,1.257090,1.039088,0.655186,-1.289855,6.199509,-2.538851,-3.147117,-2.409928,-0.482303,1.337103,-2.566002,-1.331840,0.0,-0.649760,...,-0.639394,-1.604926,1.818004,-2.336524,-1.285978,-0.326265,-1.257942,0.406640,-1.109216,-1.235593,-0.114718,-0.309510,-0.358604,-0.268309,0.509665,0.485277,-1.822126,-0.136512,2,1,13.933237,-0.464671,-0.32638,-0.041399,-0.105891,-0.071695,-0.168464,-0.173161,-0.16849,-0.049726,-0.052151,-0.108606,-0.079622,-0.06164,-0.041126,-0.064922,-0.014144,-0.014144,-0.011768,-0.010946
3,-0.950808,-0.371718,-0.343246,1.658171,0.445173,-0.402467,-0.199834,-0.187036,-0.09259,0.42403,1.735290,-0.559949,-0.595522,-0.923516,0.341423,0.573809,-0.193247,-0.341606,-0.317357,-0.202524,0.637262,0.007444,-0.029644,-0.492591,1.112761,-0.508230,0.201002,0.188873,0.898740,0.591996,0.231832,0.257471,1.995390,1.635483,-2.295764,-0.747885,-0.242987,-1.331840,0.0,0.162461,...,-1.347275,0.549401,-0.484271,-0.423594,0.404353,-0.060591,0.337487,1.341960,0.192108,-0.506673,-0.114718,1.549122,-0.440752,0.709810,-1.090316,-0.493828,-0.774445,-0.136512,3,1,-0.166375,0.560810,-0.32638,-0.041399,-0

,p1_has_alakazam,p1_has_golem,p1_has_slowbro,p1_has_zapdos,p1_has_snorlax,p1_has_cloyster,p1_has_dragonite,p1_has_persian,p1_has_charizard,p1_has_chansey,p1_has_gengar,p1_has_jynx,p1_has_rhydon,p1_has_starmie,p1_has_tauros,p1_has_exeggutor,p1_has_articuno,p1_has_lapras,p1_has_jolteon,p1_has_victreebel,p1_mean_hp,p1_mean_spe,p1_mean_atk,p1_mean_def,p1_mean_special,p1_lead_hp,p1_lead_atk,p1_lead_def,p1_lead_special,p1_lead_spe,p2_lead_hp,p2_lead_spe,p2_lead_atk,p2_lead_def,p2_lead_special,spe_lead_adv,p1_unique_pokemon,p2_unique_pokemon,n_turns,mean_damage_dealt,...,p2_pkm_par,p1_pkm_noeffect,p1_pkm_reflect,p2_pkm_noeffect,p1_mean_power,p1_mean_accuracy,p1_lost_hp,p1_turns_statused,p1_missed_turns,p1_switches,p1_net_boost,p2_mean_power,p2_mean_accuracy,p2_lost_hp,p2_turns_statused,p2_missed_turns,p2_switches,p2_net_boost,battle_id,player_won,p2_pkm_substitute,p2_pkm_reflect,p1_pkm_frz,p1_pkm_wrap,p1_pkm_tox,p1_pkm_psn,p1_pkm_confusion,p1_pkm_substitute,p2_pkm_confusion,p1_pkm_clamp,p2_pkm_clamp,p2_pkm_tox,p2_pkm_psn,p2_pkm_brn,p2_pkm_wrap,p1_pkm_brn,p1_pkm_typechange,p2_pkm_typechange,p1_pkm_firespin,p2_pkm_firespin
count,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.000000,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.0,1.000000e+04,...,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.00000,10000.000000,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
mean,-7.105427e-18,4.973799e-17,4.831691e-17,-1.421085e-18,-4.405365e-17,4.121148e-17,-7.105427e-17,4.831691e-17,-1.421085e-17,-7.531753e-17,3.090861e-17,6.394885e-18,1.016076e-16,0.000000,1.875833e-16,9.805490e-17,1.278977e-17,4.973799e-18,7.673862e-17,-3.126388e-17,-4.185097e-16,-9.471535e-16,7.389644e-17,3.140599e-16,1.675460e-15,-3.765876e-17,3.268497e-17,-3.268497e-17,2.422951e-16,1.854517e-16,5.115908e-17,1.605827e-16,1.012523e-16,7.105427e-19,2.785328e-16,-3.126388e-17,-1.136868e-16,-2.842171e-16,0.0,5.909939e-16,...,-7.389644e-17,3.147704e-16,-2.131628e-18,9.308110e-17,-1.548983e-16,1.150369e-15,-2.160050e-16,1.136868e-17,-1.364242e-16,-5.684342e-17,-2.842171e-18,1.620037e-16,-1.056577e-15,-4.547474e-17,-9.094947e-17,1.364242e-16,3.183231e-16,-2.273737e-17,4999.50000,0.500000,1.563194e-17,-1.083578e-17,-2.842171e-18,2.131628e-18,2.273737e-17,-1.989520e-17,-2.557954e-17,1.421085e-18,-5.542233e-17,-7.460699e-18,-2.060574e-17,9.947598e-18,-2.842171e-18,-3.179679e-17,6.394885e-18,-1.634248e-17,1.030287e-17,1.030287e-17,3.552714e-19,6.394885e-18
std,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,0.0,1.000050e+00,...,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,2886.89568

p1_has_alakazam      float64
p1_has_golem         float64
p1_has_slowbro       float64
p1_has_zapdos        float64
p1_has_snorlax       float64
                      ...   
p1_pkm_brn           float64
p1_pkm_typechange    float64
p2_pkm_typechange    float64
p1_pkm_firespin      float64
p2_pkm_firespin      float64
Length: 118, dtype: object


Testing dataset preview:


,p1_has_alakazam,p1_has_slowbro,p1_has_zapdos,p1_has_snorlax,p1_has_cloyster,p1_has_articuno,p1_has_dragonite,p1_has_persian,p1_has_charizard,p1_has_chansey,p1_has_gengar,p1_has_jynx,p1_has_rhydon,p1_has_starmie,p1_has_tauros,p1_has_exeggutor,p1_has_golem,p1_has_lapras,p1_has_jolteon,p1_has_victreebel,p1_mean_hp,p1_mean_spe,p1_mean_atk,p1_mean_def,p1_mean_special,p1_lead_hp,p1_lead_atk,p1_lead_def,p1_lead_special,p1_lead_spe,p2_lead_hp,p2_lead_spe,p2_lead_atk,p2_lead_def,p2_lead_special,spe_lead_adv,p1_unique_pokemon,p2_unique_pokemon,n_turns,mean_damage_dealt,...,p1_pkm_noeffect,p2_pkm_noeffect,p1_mean_power,p1_mean_accuracy,p1_lost_hp,p1_turns_statused,p1_missed_turns,p1_switches,p1_net_boost,p2_mean_power,p2_mean_accuracy,p2_lost_hp,p2_turns_statused,p2_missed_turns,p2_switches,p2_net_boost,battle_id,p2_pkm_frz,p1_pkm_wrap,p2_pkm_clamp,p2_pkm_wrap,p1_pkm_brn,p1_pkm_confusion,p2_pkm_fnt,p1_pkm_clamp,p1_pkm_tox,p1_pkm_psn,p2_pkm_reflect,p1_pkm_reflect,p2_pkm_substitute,p2_pkm_tox,p2_pkm_brn,p2_pkm_confusion,p2_pkm_psn,p1_pkm_substitute,p1_pkm_firespin,p2_pkm_firespin,p1_pkm_typechange,p2_pkm_typechange,p1_pkm_disable
0,-0.95616,-0.354428,-0.601687,0.459609,-0.402132,-0.202524,-0.204655,5.101669,-0.101514,0.438707,1.687932,-0.541596,-0.590593,-0.936041,0.353354,0.558562,-0.362245,-0.348682,-0.316784,-0.192689,0.347635,0.297212,-0.497947,-0.937347,-0.327487,2.427470,-1.903090,-1.690965,-0.344096,-1.460522,-0.111797,1.206605,0.120196,0.067841,-0.216179,-0.763728,-0.252920,-0.210231,0.0,0.358902,...,0.544853,0.521117,-0.069245,0.803228,0.429182,1.155070,1.166160,-0.143189,-0.134063,0.672315,-0.350403,0.325505,1.010900,-0.172866,-0.403120,-0.130808,0,-0.317795,-0.051956,-0.049493,-0.044594,-0.060414,-0.168902,-0.602802,-0.046761,-0.105941,-0.081073,-0.450774,-0.480383,-0.160492,-0.113318,-0.071229,-0.179306,-0.072233,-0.145773,-0.014144,-0.014144,-0.014144,-0.014144,0.0
1,-0.95616,-0.354428,-0.601687,-2.175763,2.486746,-0.202524,4.886269,-0.196014,-0.101514,-2.279427,-0.592441,-0.541596,-0.590593,1.068329,0.353354,-1.790311,-0.362245,-0.348682,3.156725,5.189698,-3.135315,2.448247,2.499830,2.635469,-0.209235,-0.435025,0.209725,0.193802,-0.096794,1.272235,-0.437793,0.730007,-0.697689,-0.607419,1.098777,1.309367,-1.414171,0.926151,0.0,-1.754321,...,0.544853,0.521117,-0.312162,-2.037690,-1.858883,-1.600432,-1.116337,-1.964933,-0.134063,1.285556,1.153320,-0.262702,0.313102,3.749966,-0.054702,-0.130808,1,-0.317795,-0.051956,-0.049493,-0.044594,-0.060414,-0.168902,-0.602802,-0.046761,-0.105941,-0.081073,-0.450774,-0.480383,-0.160492,-0.113318,-0.071229,-0.179306,-0.072233,-0.145773,-0.014144,-0.014144,-0.014144,-0.014144,0.0
2,-0.95616,-0.354428,-0.601687,0.459609,2.486746,-0.202524,-0.204655,-0.196014,-0.101514,0.438707,-0.592441,-0.541596,1.693215,-0.936041,0.353354,-1.790311,-0.362245,-0.348682,-0.316784,5.189698,0.531594,-1.751393,1.825911,1.784799,-2.456024,2.427470,-1.903090,-1.690965,-0.344096,-1.460522,-0.437793,0.730007,-0.697689,-0.607419,1.098777,-0.763728,-0.252920,0.926151,0.0,-2.355355,...,0.073364,-1.647394,-1.296566,-3.555448,-1.430696,-2.151533,-0.464195,-1.600585,-0.134063,-0.480580,-0.290254,-1.185299,1.010900,-0.499769,-0.403120,1.240344,2,1.906624,3.606893,3.005633,16.943684,-0.060414,-0.168902,-0.602802,-0.046761,-0.105941,-0.081073,-0.450774,-0.480383,-0.160492,-0.113318,-0.071229,-0.179306,-0.072233,-0.145773,-0.014144,-0.014144,-0.014144,-0.014144,0.0
3,-0.95616,-0.354428,-0.601687,0.459609,2.486746,-0.202524,-0.204655,-0.196014,-0.101514,0.438707,-0.592441,1.846394,-0.590593,-0.936041,-2.830019,0.558562,-0.362245,-0.348682,3.156725,-0.192689,0.102357,-0.522230,-1.078912,0.083458,0.263773,-0.435025,-0.318479,-0.662910,-0.838700,0.076654,2.985166,-3.559383,2.573852,0.292928,-2.583099,1.309367,-2.575421,-0.210231,0.0,0.457355,...,-0.162381,0.521117,-1.357988,0.803228,0.224281,1.155070,-1.442408,-1.600585,-1.398808,0.683265,0.707772,0.256305,1.010900,-0.172866,-1.448375,-0.130808,3,-0.317795,-0.051956,-0.049493,-0.0

,p1_has_alakazam,p1_has_slowbro,p1_has_zapdos,p1_has_snorlax,p1_has_cloyster,p1_has_articuno,p1_has_dragonite,p1_has_persian,p1_has_charizard,p1_has_chansey,p1_has_gengar,p1_has_jynx,p1_has_rhydon,p1_has_starmie,p1_has_tauros,p1_has_exeggutor,p1_has_golem,p1_has_lapras,p1_has_jolteon,p1_has_victreebel,p1_mean_hp,p1_mean_spe,p1_mean_atk,p1_mean_def,p1_mean_special,p1_lead_hp,p1_lead_atk,p1_lead_def,p1_lead_special,p1_lead_spe,p2_lead_hp,p2_lead_spe,p2_lead_atk,p2_lead_def,p2_lead_special,spe_lead_adv,p1_unique_pokemon,p2_unique_pokemon,n_turns,mean_damage_dealt,...,p1_pkm_noeffect,p2_pkm_noeffect,p1_mean_power,p1_mean_accuracy,p1_lost_hp,p1_turns_statused,p1_missed_turns,p1_switches,p1_net_boost,p2_mean_power,p2_mean_accuracy,p2_lost_hp,p2_turns_statused,p2_missed_turns,p2_switches,p2_net_boost,battle_id,p2_pkm_frz,p1_pkm_wrap,p2_pkm_clamp,p2_pkm_wrap,p1_pkm_brn,p1_pkm_confusion,p2_pkm_fnt,p1_pkm_clamp,p1_pkm_tox,p1_pkm_psn,p2_pkm_reflect,p1_pkm_reflect,p2_pkm_substitute,p2_pkm_tox,p2_pkm_brn,p2_pkm_confusion,p2_pkm_psn,p1_pkm_substitute,p1_pkm_firespin,p2_pkm_firespin,p1_pkm_typechange,p2_pkm_typechange,p1_pkm_disable
count,5.000000e+03,5000.000000,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5000.0,5.000000e+03,...,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5000.000000,5000.000000,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5000.000000,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5000.000000
mean,-5.684342e-17,0.000000,1.136868e-17,-2.273737e-17,-8.526513e-17,1.705303e-17,3.126388e-17,4.405365e-17,1.705303e-17,1.847411e-17,4.263256e-18,-3.126388e-17,2.806644e-17,-6.821210e-17,-1.250555e-16,2.842171e-17,-8.526513e-17,-2.771117e-17,-5.684342e-18,-5.684342e-17,-9.123369e-16,1.355005e-15,3.637979e-16,-7.560175e-16,-1.540457e-15,-8.526513e-17,1.421085e-16,-9.947598e-18,3.396394e-16,-2.415845e-17,-4.831691e-17,-3.467449e-16,-2.273737e-17,-9.166001e-17,1.634248e-17,-3.126388e-17,-5.002221e-16,4.774847e-16,0.0,3.115730e-16,...,-1.421085e-16,2.557954e-16,3.836931e-17,1.720935e-15,4.547474e-16,1.364242e-16,-1.818989e-16,-2.501110e-16,2.273737e-17,3.581135e-16,-1.016076e-15,9.094947e-17,4.547474e-17,-1.136868e-16,-2.273737e-17,3.410605e-17,2499.500000,0.000000,-1.278977e-17,4.263256e-18,-5.684342e-18,-1.421085e-17,-1.421085e-17,2.557954e-17,-1.278977e-17,-1.136868e-17,5.684342e-18,5.258016e-17,1.136868e-17,7.105427e-18,0.000000,8.526513e-18,-3.410605e-17,-1.136868e-17,2.273737e-17,7.105427e-18,9.947598e-18,1.421085e-18,1.421085e-18,0.001200
std,1.000100e+00,1.000100,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,0.0,1.000100e+00,...,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1443.520003,1.000100,1.000100e+00,1.000100e

p1_has_alakazam      float64
p1_has_slowbro       float64
p1_has_zapdos        float64
p1_has_snorlax       float64
p1_has_cloyster      float64
                      ...   
p1_pkm_firespin      float64
p2_pkm_firespin      float64
p1_pkm_typechange    float64
p2_pkm_typechange    float64
p1_pkm_disable       float64
Length: 118, dtype: object

['p1_has_alakazam', 'p1_has_slowbro', 'p1_has_zapdos', 'p1_has_snorlax', 'p1_has_cloyster', 'p1_has_articuno', 'p1_has_dragonite', 'p1_has_persian', 'p1_has_charizard', 'p1_has_chansey', 'p1_has_gengar', 'p1_has_jynx', 'p1_has_rhydon', 'p1_has_starmie', 'p1_has_tauros', 'p1_has_exeggutor', 'p1_has_golem', 'p1_has_lapras', 'p1_has_jolteon', 'p1_has_victreebel', 'p1_mean_hp', 'p1_mean_spe', 'p1_mean_atk', 'p1_mean_def', 'p1_mean_special', 'p1_lead_hp', 'p1_lead_atk', 'p1_lead_def', 'p1_lead_special', 'p1_lead_spe', 'p2_lead_hp', 'p2_lead_spe', 'p2_lead_atk', 'p2_lead_def', 'p2_lead_special', 'spe_lead_adv', 'p1_unique_pokemon', 'p2_unique_pokemon', 'n_turns', 'mean_damage_dealt', 'final_p1_hp', 'p1_fainted_count', 'p2_fainted_count', 'p1_mean_final_hp', 'p2_mean_final_hp', 'p1_hp_loss_ratio', 'p2_hp_loss_ratio', 'hp_diff_lost', 'hp_diff_final', 'hp_ratio_p1_p2', 'p1_mean_hp_pct', 'p2_mean_hp_pct', 'p1_min_hp', 'p2_min_hp', 'p1_survival_ratio', 'p2_survival_ratio', 'survival_diff', 'p1_av

### 3. Training Models

In [151]:
# Define predictor features (X) and target (y)
X_train = train_df[keepers] #[keepers]
print(len(train_df.columns))
y_train = train_df['player_won']

X_test = test_df[keepers] #[keepers]

print("Training...")
model = XGBClassifier(
    random_state=100,
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3,
    eval_metric='logloss',
    n_jobs=-1
)
model = LGBMClassifier(
    learning_rate=0.03165579861591166, 
    num_leaves=125,
    max_depth=3, 
    min_child_samples=29, 
    reg_alpha=7.1991672853117725,  
    reg_lambda=3.052954302286083, 
    feature_fraction=0.9504554803940148, 
    bagging_fraction=0.9731871954622454, 
    bagging_freq=6, 
    objective='binary',
    n_estimators=335,
    random_state=100,
    verbose=-1
)
model.fit(X_train, y_train)
print("Model training complete.")

118
Training...
Model training complete.


In [152]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=100)
cv_results = cross_validate(
    model,
    X_train,
    y_train,
    cv=cv,
    scoring={
        "accuracy_score": make_scorer(accuracy_score),
        "precision_score": make_scorer(precision_score),
        "recall_score": make_scorer(recall_score),
        "f1_score": make_scorer(f1_score),
        "roc_auc_score": make_scorer(roc_auc_score)
    },
    return_train_score=True,
    n_jobs=-1
)

results_df = pd.DataFrame(cv_results)
display(results_df)

results_df = pd.DataFrame(cv_results)
summary = results_df.filter(regex='(train_|test_)').describe().loc[['mean', 'std']].T
summary.rename(columns={'mean': 'Mean', 'std': 'Std'}, inplace=True)
display(summary)

,fit_time,score_time,test_accuracy_score,train_accuracy_score,test_precision_score,train_precision_score,test_recall_score,train_recall_score,test_f1_score,train_f1_score,test_roc_auc_score,train_roc_auc_score
0,8.247918,0.099574,0.821,0.852778,0.822938,0.856022,0.818,0.848222,0.820461,0.852104,0.821,0.852778
1,7.132856,0.059538,0.822,0.851000,0.828571,0.854069,0.812,0.846667,0.820202,0.850352,0.822,0.851000
2,7.045014,0.049572,0.830,0.851111,0.834008,0.853626,0.824,0.847556,0.828974,0.850580,0.830,0.851111
3,6.545305,0.090527,0.824,0.851000,0.817647,0.853277,0.834,0.847778,0.825743,0.850518,0.824,0.851000
4,8.222096,0.043661,0.835,0.850778,0.829077,0.854639,0.844,0.845333,0.836472,0.849961,0.835,0.850778
5,8.605643,0.048555,0.843,0.850444,0.858038,0.853271,0.822,0.846444,0.839632,0.849844,0.843,0.850444
6,6.401460,0.047755,0.838,0.852000,0.844898,0.853256,0.828,0.850222,0.836364,0.851736,0.838,0.852000
7,6.412069,0.051479,0.854,0.847778,0.864198,0.851370,0.840,0.842667,0.851927,0.846996,0.854,0.847778
8,2.772865,0.041168,0.847,0.848111,0.849095,0.850056,0.844,0.845333,0.846540,0.847688,0.847,0.848111
9,2.724530,0.039760,0.846,0.849667,0.843254,0.852092,0.850,0.846222,0.846614,0.849147,0.846,0.849667


,Mean,Std
test_accuracy_score,0.836000,0.011547
train_accuracy_score,0.850467,0.001570
test_precision_score,0.839172,0.015268
train_precision_score,0.853168,0.001681
test_recall_score,0.831600,0.012712
train_recall_score,0.846644,0.002024
test_f1_score,0.835293,0.011213
train_f1_score,0.849893,0.001605
test_roc_auc_score,0.836000,0.011547
train_roc_auc_score,0.850467,0.001570


### 4. Creating the Submission File

The competition requires a `.csv` file with two columns: `battle_id` and `player_won`. Let's use our trained model to make predictions on the test set and format them correctly.

In [153]:
print("Generating predictions on the test set...")
submission_df = pd.DataFrame({
    'battle_id': test_df['battle_id'],
    'player_won': model.predict(X_test)
})

submission_df.to_csv('submission.csv', index=False)

print("\n'submission.csv' file created successfully!")
display(submission_df.head())

Generating predictions on the test set...

'submission.csv' file created successfully!


,battle_id,player_won
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1


### 5. Submitting Your Results

Once you have generated your `submission.csv` file, there are two primary ways to submit it to the competition.

---

#### Method A: Submitting Directly from the Notebook

This is the standard method for code competitions. It ensures that your submission is linked to the code that produced it, which is crucial for reproducibility.

1.  **Save Your Work:** Click the **"Save Version"** button in the top-right corner of the notebook editor.
2.  **Run the Notebook:** In the pop-up window, select **"Save & Run All (Commit)"** and then click the **"Save"** button. This will run your entire notebook from top to bottom and save the output, including your `submission.csv` file.
3.  **Go to the Viewer:** Once the save process is complete, navigate to the notebook viewer page. 
4.  **Submit to Competition:** In the viewer, find the **"Submit to Competition"** section. This is usually located in the header of the output section or in the vertical "..." menu on the right side of the page. Clicking the **Submit** button this will submit your generated `submission.csv` file.

After submitting, you will see your score in the **"Submit to Competition"** section or in the [Public Leaderboard](https://www.kaggle.com/competitions/fds-pokemon-battles-prediction-2025/leaderboard?).

---

#### Method B: Manual Upload

You can also generate your predictions and submission file using any environment you prefer (this notebook, Google Colab, or your local machine).

1.  **Generate the `submission.csv` file** using your model.
2.  **Download the file** to your computer.
3.  **Navigate to the [Leaderboard Page](https://www.kaggle.com/competitions/fds-pokemon-battles-prediction-2025/leaderboard?)** and click on the **"Submit Predictions"** button.
4.  **Upload Your File:** Drag and drop or select your `submission.csv` file to upload it.

This method is quick, but keep in mind that for the final evaluation, you might be required to provide the code that generated your submission.

Good luck!